In [1]:
!pip install requests_html
from time import sleep
import numpy as np
import requests
import os
from bs4 import BeautifulSoup
import json
import pandas as pd
from google.colab import drive 
import math
from requests_html import HTMLSession  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 13.4 MB/s eta 0:00:00
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=e43b0302c5bbd7c06dd358a494ee6b536ae485b1f1d663c50f1e07fa3bd9de59
  Stored in directory: /root/.cache/pip/wheels/e5/33/1f/68392720485b3ecf125a69e700baaab7624616deedea2fa6e2
Successfully built parse
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
drive.mount('/content/drive')
df_query = pd.read_csv('/content/drive/MyDrive/Proyecto App Boardgames/Web scraper/test.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def find_bgg_id(boardgame):
  URL_busqueda = f'https://boardgamegeek.com/geeksearch.php?action=search&advsearch=1&objecttype=boardgame&q={boardgame}'
  URL = URL_busqueda
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, "html.parser")
  pagina = soup.find(id='collectionitems')
  juego = pagina.find_all("div", id="results_objectname1")
  if juego:
    bgg_id = juego[0].find("a", class_="primary").get('href').split('"')[0]
  else:
    bgg_id = np.nan 
  return bgg_id

In [ ]:
df_bgg = pd.read_csv('/content/drive/MyDrive/Proyecto App Boardgames/Web scraper/bgg_dataset.csv', delimiter=';')
df_boardgames = pd.DataFrame([], columns=['bgg_id',
                                          'bgg_name',
                                          'year',
                                          'store_name',
                                          'alternative_names',
                                          'min_players',
                                          'max_players',
                                          'play_time',
                                          'min_age',
                                          'users_rated',
                                          'rating_average',
                                          'bgg_rank',
                                          'complexity_average',
                                          'owned_users',
                                          'mechanics',
                                          'domains'])
df_boardgames = df_boardgames.append(df_bgg)
df_boardgames['alternative_names'] = ""
df_boardgames['store_name'] = ""
df_boardgames.loc[df_boardgames['bgg_id'] == 9209, 'alternative_names'] = "Aventureros al Tren America"

In [ ]:
boardgame_list = list(df_query['name'])
for boardgame in boardgame_list:
  bgg_id = df_boardgames[df_boardgames['alternative_names'].str.contains(boardgame, case=False)]['bgg_id']
  if bgg_id.empty:
    bgg_id = find_bgg_id(boardgame)
    if isinstance(bgg_id,float):
      if math.isnan(bgg_id):
        print(boardgame)
        continue
        #ACA RELLENAR LA BASE DE DATOS df_boardgames CON UNA ENTRADA NUEVA
        #SIN DATOS PARA LA BASE DE DATOS DE BGG

    bgg_id = int(bgg_id.split("/")[2])
    if df_boardgames[df_boardgames['bgg_id'] == bgg_id].empty:
      #NO EXISTE LA ENTRADA EN LA BASE DE DATOS, PERO SI EN BGG.
      #HAY QUE CREAR UNA NUEVA ENTRADA
      continue  

    names = df_boardgames.loc[df_boardgames['bgg_id'] == bgg_id]['alternative_names']
    #if names.empty:
    #  df_boardgames.loc[df_boardgames['bgg_id'] == bgg_id, 'alternative_names'] = boardgame
    #else:
    #  df_boardgames.loc[df_boardgames['bgg_id'] == bgg_id, 'alternative_names'] = names.values[0]+";"+boardgame
  else:
    bgg_id = int(bgg_id)
  sleep(1)

True

In [ ]:
df_boardgames[df_boardgames['alternative_names']!=""]

,bgg_id,bgg_name,year,store_name,alternative_names,min_players,max_players,play_time,min_age,users_rated,rating_average,bgg_rank,complexity_average,owned_users,mechanics,domains
173,9209.0,Ticket to Ride,2004.0,,Aventureros al Tren America,2,5,60,8,71611,"7,42",174,"1,85",97463.0,"Card Drafting, End Game Bonuses, Hand Manageme...",Family Games


In [ ]:
def find_bgg_info(bgg_id):
  bgg_info = pd.DataFrame([], columns=['bgg_id',
                                      'bgg_name',
                                      'year',
                                      'min_players',
                                      'max_players',
                                      'play_time',
                                      'min_age',
                                      'users_rated',
                                      'rating_average',
                                      'bgg_rank',
                                      'complexity_average',
                                      'owned_users',
                                      'mechanics',
                                      'domains'])
  URL_busqueda = f'https://boardgamegeek.com/boardgame/{bgg_id}/boardgame/credits'
  URL = URL_busqueda
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, "html.parser")
  pagina = soup.find("main", id='mainbody')
  print(pagina)
  #print(scripts)
  #info = pagina.find_all("div", id="game-header-title-info")
  #info.find("a", class_="ng-binding").get('href').split('"')[0]
find_bgg_info(13)

<ipython-input-11-92006cccfffe>:20: RuntimeWarning: coroutine 'HTML.arender' was never awaited
  r.html.arender()


<main class="global-body flex-grow-1" id="mainbody" ng-class="{ 'has-overlay-sidebar': layoutctrl.showOverlaySidebar, 'has-hidden-fixed-sidebar': layoutctrl.localStorage.hideFixedSidebar }">
<a id="mainbodytarget" tabindex="-1"></a>
<!-- Home King Ad -->
<div class="item-hero-layout" hide-ad-block="blockleaderboard">
<div class="advertisement-leaderboard">
<div class="center-block" ng-dfp-ad="dfp-leaderboard"></div>
</div>
</div>
<div class="global-body-content-container container-fluid">
<geekoutput-sidebar deactivate-overlay-sidebar="layoutctrl.deactivateOverlaySidebar" show-overlay-sidebar="layoutctrl.showOverlaySidebar"></geekoutput-sidebar>
<div class="global-body-content pending" ng-class="{'ready': layoutctrl.ready}">
<a id="maincontenttarget" tabindex="-1"></a>
<div class="support-plea-container" hide-ad-block="blocksupportdrive">
<geekoutput-plea></geekoutput-plea>
</div>
<script src="https://cdnjs.cloudflare.com/ajax/libs/highcharts/8.2.2/highstock.src.min.js"></script>
<scri